In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np

In [17]:
telco_df = pd.read_excel("../datos/traintelco.xlsx")

In [3]:
telco_df.head()

,id,Fecha de nacimiento,tipo cliente,Factura online,Antigüedad Equipo,Plan de datos,facturación,mora,Fecha inicio contrato,minutos,resultado
0,5,1990-05-01 00:05:22.046,2,1,9,0,44588,34,2017-12-22 10:09:47.504,644,0
1,8,1980-06-30 19:08:57.680,2,0,41,1,341480,3,2017-07-28 02:54:52.022,663,0
2,9,1984-10-10 21:23:28.869,2,0,38,1,230662,20,2017-06-29 08:15:47.525,1039,1
3,11,1986-07-01 08:58:42.308,2,1,11,1,205336,9,2017-03-13 18:39:25.308,535,0
4,16,1986-01-23 00:26:52.782,1,1,6,0,429525,0,2015-08-02 21:50:40.258,1869,0


In [21]:
telco_df.rename(columns={'Fecha de nacimiento': 'fecha_nacimiento'}, inplace=True)
telco_df.rename(columns={'tipo cliente': 'tipo_cliente'}, inplace=True)
telco_df.rename(columns={'Factura online': 'factura_online'}, inplace=True)
telco_df.rename(columns={'Antigüedad Equipo': 'antiguedad_equipo'}, inplace=True)
telco_df.rename(columns={'facturación': 'facturacion'}, inplace=True)
telco_df.rename(columns={'Fecha inicio contrato': 'fecha_inicio_contrato'}, inplace=True)
telco_df.rename(columns={'Plan de datos': 'plan_datos'}, inplace=True)
fecha_actual = datetime(2019, 1, 1)
# Calcular la diferencia de años
telco_df['edad'] = (fecha_actual - telco_df['fecha_nacimiento']).astype('<m8[Y]').astype(int)
telco_df['antiguedad'] = (fecha_actual - telco_df['fecha_inicio_contrato']).dt.days
telco_df = telco_df.drop('fecha_nacimiento',axis=1)
telco_df = telco_df.drop('fecha_inicio_contrato',axis=1)
telco_df = telco_df.drop('id',axis=1)



In [22]:
telco_df

,tipo_cliente,factura_online,antiguedad_equipo,plan_datos,facturacion,mora,minutos,resultado,edad,antiguedad
0,2,1,9,0,44588,34,644,0,28,374
1,2,0,41,1,341480,3,663,0,38,521
2,2,0,38,1,230662,20,1039,1,34,550
3,2,1,11,1,205336,9,535,0,32,658
4,1,1,6,0,429525,0,1869,0,32,1247
...,...,...,...,...,...,...,...,...,...,...
8238,2,1,7,1,196444,27,8894,1,57,758
8239,1,1,13,0,257538,3,662,0,56,631
8240,2,1,10,1,288037,41,1465,0,57,363
8241,1,0,33,1,207627,3,494,0,71,510


In [23]:
# creating features and label 

from tensorflow.keras.utils import to_categorical

X = telco_df.drop('resultado', axis = 1)
y = telco_df.resultado


In [24]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority')
x_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

0    6882
1    6882
Name: resultado, dtype: int64

In [25]:
condiciones = [
    (x_sm['tipo_cliente'] == 1),
    (x_sm['tipo_cliente'] == 2),
    (x_sm['tipo_cliente'] == 3)
]

valores = ['Hombre', 'Mujer', 'Empresa']

x_sm['tipo_cliente_2'] = np.select(condiciones, valores, default='Desconocido')
x_sm= x_sm.drop('tipo_cliente',axis=1)

condiciones2 = [
    (x_sm['factura_online'] == 0),
    (x_sm['factura_online'] == 1)
]

valores2 = ['si', 'no']

x_sm['factura_online_2'] = np.select(condiciones2, valores2, default='Desconocido')
x_sm=x_sm.drop('factura_online',axis=1)

condiciones3 = [
    (x_sm['plan_datos'] == 0),
    (x_sm['plan_datos'] == 1)
]

valores3 = ['si', 'no']

x_sm['premium_datos'] = np.select(condiciones3, valores3, default='Desconocido')
x_sm=x_sm.drop('plan_datos',axis=1)
x_sm = pd.get_dummies(x_sm)
x_sm.head()

,antiguedad_equipo,facturacion,mora,minutos,edad,antiguedad,tipo_cliente_2_Empresa,tipo_cliente_2_Hombre,tipo_cliente_2_Mujer,factura_online_2_no,factura_online_2_si,premium_datos_no,premium_datos_si
0,9,44588,34,644,28,374,0,0,1,1,0,0,1
1,41,341480,3,663,38,521,0,0,1,0,1,1,0
2,38,230662,20,1039,34,550,0,0,1,0,1,1,0
3,11,205336,9,535,32,658,0,0,1,1,0,1,0
4,6,429525,0,1869,32,1247,0,1,0,1,0,0,1


In [56]:
x_sm.to_csv('datos_prueba_sin_estandarizar.csv',index = False)

In [26]:
# splitting data into training set and test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_sm, y_sm, test_size = 0.2)

In [27]:
# Scaling data

from sklearn.preprocessing import StandardScaler

# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

X_train=(X_train-np.min(X_train))/(np.max(X_train)-np.min(X_train)).values
X_test=(X_test-np.min(X_test))/(np.max(X_test)-np.min(X_test)).values

C:\Users\javier-pc\anaconda3\envs\maestria_analitica\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
C:\Users\javier-pc\anaconda3\envs\maestria_analitica\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
C:\Users\javier-pc\anaconda3\envs\maestria_analitica\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, 

In [28]:
X_train

,antiguedad_equipo,facturacion,mora,minutos,edad,antiguedad,tipo_cliente_2_Empresa,tipo_cliente_2_Hombre,tipo_cliente_2_Mujer,factura_online_2_no,factura_online_2_si,premium_datos_no,premium_datos_si
11194,0.638298,0.618267,0.084746,0.119189,0.730769,0.403026,0.0,0.0,1.0,0.0,1.0,1.0,0.0
7408,0.872340,0.544528,0.305085,0.069485,0.576923,0.149728,0.0,0.0,1.0,0.0,1.0,1.0,0.0
9155,0.276596,0.559934,0.033898,0.055274,0.615385,0.498836,1.0,0.0,0.0,0.0,1.0,1.0,0.0
12965,0.553191,0.407285,0.203390,0.050918,0.653846,0.262995,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3909,0.085106,0.713640,0.000000,0.134186,0.692308,0.305663,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12990,0.702128,0.535112,0.415254,0.118403,0.711538,0.223041,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1605,0.234043,0.381295,0.076271,0.071128,0.750000,0.219162,0.0,0.0,1.0,1.0,0.0,1.0,0.0
7738,0.212766,0.536981,0.355932,0.018782,0.692308,0.232739,1.0,0.0,0.0,1.0,0.0,1.0,0.0
752,0.808511,0.493572,0.000000,0.082982,0.673077,0.369279,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [29]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization

# initializing ann
model = Sequential()

# adding the first input layer and the first hidden layer
model.add(Dense(10, kernel_initializer = 'normal', activation = 'relu', input_shape = (13, )))

# adding batch normalization and dropout layer
model.add(Dropout(rate = 0.1))
model.add(BatchNormalization())

# adding the third hidden layer
model.add(Dense(7, kernel_initializer = 'normal', activation = 'relu'))

# adding batch normalization and dropout layer
model.add(Dropout(rate = 0.1))
model.add(BatchNormalization())

# adding the output layer
model.add(Dense(1, kernel_initializer = 'normal', activation = 'sigmoid'))

# compiling the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# fitting the model to the training set 

model_history = model.fit(X_train, y_train, validation_split = 0.20, validation_data = (X_test, y_test), epochs = 100)

Epoch 1/100
345/345 [==============================] - 3s 4ms/step - loss: 0.6152 - accuracy: 0.6570 - val_loss: 0.6594 - val_accuracy: 0.6240
Epoch 2/100
345/345 [==============================] - 1s 3ms/step - loss: 0.5602 - accuracy: 0.7087 - val_loss: 0.5391 - val_accuracy: 0.7356
Epoch 3/100
345/345 [==============================] - 1s 4ms/step - loss: 0.5569 - accuracy: 0.7104 - val_loss: 0.5477 - val_accuracy: 0.7218
Epoch 4/100
345/345 [==============================] - 2s 5ms/step - loss: 0.5520 - accuracy: 0.7158 - val_loss: 0.5150 - val_accuracy: 0.7465
Epoch 5/100
345/345 [==============================] - 2s 5ms/step - loss: 0.5460 - accuracy: 0.7206 - val_loss: 0.5152 - val_accuracy: 0.7436
Epoch 6/100
345/345 [==============================] - 2s 5ms/step - loss: 0.5463 - accuracy: 0.7255 - val_loss: 0.5163 - val_accuracy: 0.7490
Epoch 7/100
345/345 [==============================] - 2s 7ms/step - loss: 0.5380 - accuracy: 0.7244 - val_loss: 0.4951 - val_accuracy: 0.7494

Epoch 58/100
345/345 [==============================] - 1s 4ms/step - loss: 0.5135 - accuracy: 0.7394 - val_loss: 0.4784 - val_accuracy: 0.7606
Epoch 59/100
345/345 [==============================] - 1s 4ms/step - loss: 0.5167 - accuracy: 0.7372 - val_loss: 0.4853 - val_accuracy: 0.7566
Epoch 60/100
345/345 [==============================] - 1s 4ms/step - loss: 0.5121 - accuracy: 0.7359 - val_loss: 0.4695 - val_accuracy: 0.7679
Epoch 61/100
345/345 [==============================] - 1s 4ms/step - loss: 0.5153 - accuracy: 0.7433 - val_loss: 0.4850 - val_accuracy: 0.7552
Epoch 62/100
345/345 [==============================] - 1s 4ms/step - loss: 0.5125 - accuracy: 0.7454 - val_loss: 0.4742 - val_accuracy: 0.7610
Epoch 63/100
345/345 [==============================] - 1s 4ms/step - loss: 0.5101 - accuracy: 0.7450 - val_loss: 0.4686 - val_accuracy: 0.7693
Epoch 64/100
345/345 [==============================] - 1s 4ms/step - loss: 0.5110 - accuracy: 0.7441 - val_loss: 0.4709 - val_accuracy:

In [144]:
model.save('Churn_Modelling.h5')

C:\Users\javier-pc\anaconda3\envs\maestria_analitica\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:
acc = model.evaluate(X_test, y_test)[1]

print(f'Accuracy of model is {acc}')

87/87 [==============================] - 0s 3ms/step - loss: 0.4755 - accuracy: 0.7701
Accuracy of model is 0.7700690031051636


In [31]:
y_pred = model.predict(X_test)


87/87 [==============================] - 0s 2ms/step


In [192]:
y_pred = model.predict(X_test)

y_pred = pd.DataFrame(y_pred)
y_pred['resultado'] = '0'
y_pred.loc[y_pred[0] > 0.5,'resultado'] = '1'
y_pred = y_pred.drop(0,axis=1)
exportar = solucion['id']
exportar =  pd.concat([exportar, y_pred_solucion_df], axis=1)
exportar.to_csv('solucion14RN.csv', index=False)

108/108 [==============================] - 0s 2ms/step


In [191]:
exportar

,id,resultado
0,13,0
1,14,0
2,20,0
3,30,0
4,37,0
...,...,...
1995,18270,0
1996,18278,0
1997,18292,0
1998,18295,0


In [49]:
import sklearn
print('Area Under ROC-Curve: ', sklearn.metrics.roc_auc_score(y_test,y_pred))

Area Under ROC-Curve:  0.8273899407204126


In [57]:
solucion = pd.read_excel('../datos/testelco.xlsx')

solucion.rename(columns={'Fecha de nacimiento': 'fecha_nacimiento'}, inplace=True)
solucion.rename(columns={'tipo cliente': 'tipo_cliente'}, inplace=True)
solucion.rename(columns={'Factura online': 'factura_online'}, inplace=True)
solucion.rename(columns={'Antigüedad Equipo': 'antiguedad_equipo'}, inplace=True)
solucion.rename(columns={'facturación': 'facturacion'}, inplace=True)
solucion.rename(columns={'Fecha inicio contrato': 'fecha_inicio_contrato'}, inplace=True)
solucion.rename(columns={'Plan de datos': 'plan_datos'}, inplace=True)

fecha_actual = datetime(2019, 1, 1)

solucion['edad'] = (fecha_actual - solucion['fecha_nacimiento']).astype('<m8[Y]').astype(int)
solucion['antiguedad'] = (fecha_actual - solucion['fecha_inicio_contrato']).dt.days
solucion = solucion.drop('fecha_nacimiento',axis=1)
solucion = solucion.drop('fecha_inicio_contrato',axis=1)
# solucion = solucion.drop('id',axis=1)


X_solucion = solucion

X_solucion = X_solucion.drop('id',axis=1)

condiciones = [
    (X_solucion['tipo_cliente'] == 1),
    (X_solucion['tipo_cliente'] == 2),
    (X_solucion['tipo_cliente'] == 3)
]

valores = ['Hombre', 'Mujer', 'Empresa']

X_solucion['tipo_cliente_2'] = np.select(condiciones, valores, default='Desconocido')
X_solucion= X_solucion.drop('tipo_cliente',axis=1)

condiciones2 = [
    (X_solucion['factura_online'] == 0),
    (X_solucion['factura_online'] == 1)
]

valores2 = ['si', 'no']

X_solucion['factura_online_2'] = np.select(condiciones2, valores2, default='Desconocido')
X_solucion=X_solucion.drop('factura_online',axis=1)

condiciones3 = [
    (X_solucion['plan_datos'] == 0),
    (X_solucion['plan_datos'] == 1)
]

valores3 = ['si', 'no']

X_solucion['premium_datos'] = np.select(condiciones3, valores3, default='Desconocido')
X_solucion=X_solucion.drop('plan_datos',axis=1)

X_solucion = pd.get_dummies(X_solucion)
X_solucion.to_csv('datos_a_probar.csv',index=False)

X_solucion=(X_solucion-np.min(X_solucion))/(np.max(X_solucion)-np.min(X_solucion)).values


C:\Users\javier-pc\anaconda3\envs\maestria_analitica\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
C:\Users\javier-pc\anaconda3\envs\maestria_analitica\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
C:\Users\javier-pc\anaconda3\envs\maestria_analitica\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, 

In [49]:
# Scaling data
# X_solucion = sc.fit_transform(X_solucion)
X_solucion

,antiguedad_equipo,facturacion,mora,minutos,edad,antiguedad,tipo_cliente_2_Empresa,tipo_cliente_2_Hombre,tipo_cliente_2_Mujer,factura_online_2_no,factura_online_2_si,premium_datos_no,premium_datos_si
0,0.361702,0.582206,0.000000,0.091967,0.000000,0.328040,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,0.106383,0.603759,0.000000,0.108910,0.192982,0.375290,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,0.723404,0.436470,0.383929,0.046194,0.157895,0.158792,0.0,0.0,1.0,0.0,1.0,1.0,0.0
3,0.212766,0.415733,0.294643,0.067521,0.263158,0.112316,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.702128,0.501967,0.133929,0.127371,0.175439,0.256778,0.0,1.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.744681,0.523586,0.276786,0.036837,0.719298,0.147947,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1996,0.765957,0.602311,0.000000,0.106803,0.719298,0.365995,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1997,0.148936,0.533328,0.392857,0.031442,0.736842,0.118513,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1998,0.872340,0.738382,0.008929,0.122397,0.649123,0.252130,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [47]:
solucion

,tipo_cliente,factura_online,antiguedad_equipo,plan_datos,facturacion,mora,minutos,edad,antiguedad,tipo_cliente_2
0,2,0,18,1,319690,0,1202,22,882,Mujer
1,2,1,6,1,333812,0,1403,33,1004,Mujer
2,2,0,35,1,224201,43,659,31,445,Mujer
3,1,1,11,1,210614,33,912,37,325,Hombre
4,1,0,34,1,267116,15,1622,32,698,Hombre
...,...,...,...,...,...,...,...,...,...,...
1995,1,0,36,1,281281,31,548,63,417,Hombre
1996,1,0,37,1,332863,0,1378,63,980,Hombre
1997,2,1,8,1,287664,44,484,64,341,Mujer
1998,1,0,42,1,422019,1,1563,59,686,Hombre


In [53]:
y_pred_solucion = model.predict(X_solucion)
y_pred_solucion_df = pd.DataFrame(y_pred_solucion)
# y_pred_solucion_df['resultado'] = '0'
y_pred_solucion_df = y_pred_solucion_df.rename(columns={0: 'resultado'})

# y_pred_solucion_df.loc[y_pred_solucion_df[0] > 0.3,'resultado'] = '1'
# y_pred_solucion_df = y_pred_solucion_df.drop(0,axis=1)

exportar = solucion['id']
exportar =  pd.concat([exportar, y_pred_solucion_df], axis=1)
exportar.to_csv('solucion14RN.csv', index=False)
# exportar



63/63 [==============================] - 0s 2ms/step


In [52]:
exportar

,id,resultado
0,13,0.027229
1,14,0.063498
2,20,0.036038
3,30,0.015045
4,37,0.040340
...,...,...
1995,18270,0.347587
1996,18278,0.621435
1997,18292,0.261971
1998,18295,0.315097


In [45]:
y_pred_solucion_df

,resultado
0,0.027229
1,0.063498
2,0.036038
3,0.015045
4,0.040340
...,...
1995,0.347587
1996,0.621435
1997,0.261971
1998,0.315097
